## Create Test data for the emergency scenerio

In [1]:
import os
os.chdir('../..')
from trips.config import *
from trips.weibull_trips import main
import subprocess

In [ ]:
number_cars = 400

if not os.path.exists("test"):
    os.makedirs("test")
    

main(
    src_nodes,
    dst_nodes,
    src_prob,
    turn_prob,
    emergency_probs,
    shape,
    number_cars,
    total_timesteps,
)
subprocess.run(
    [
        "duarouter",
        "-n",
        "./network_details/intersection.net.xml",
        "-t",
        "trips.trips.xml",  
        "-o",
        f"test/intersection_Emergency.rou.xml", 
    ]
)
os.remove("trips.trips.xml")


route_files = ["test/intersection_Emergency.rou.xml"]
        

Success.up to time step: 5331.00


 Use 'route-files' instead.


In [3]:
from environment.environment import EmergencySumoEnvironment
from environment.observation import EmergencyObservationFunction
from environment.reward import emergency_reward

## Test round robin algorithm on emergency scenerio

In [ ]:
env = EmergencySumoEnvironment(
    net_file="network_details/intersection.net.xml",
    route_files=route_files,
    out_csv_name=f'outputs/test/Emergency/RoundRobin',
    min_green=15,
    yellow_time=5,
    delta_time=20,
    use_gui=False,
    num_seconds=5400,
    observation_class=EmergencyObservationFunction,
    reward_fn=emergency_reward
)

obs, info = env.reset()
done = False

action_index = 0  
while not done:
    action = action_index % 4
    
    next_obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    action_index += 1

obs, info = env.reset()
env.close()

 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 400 ACT 0 BUF 0)                  tep #4500.00 (1ms ~= 1000.00*RT, ~4000.00UPS, TraCI: 16ms, vehicles TOT 392 ACT 4 BUF 0) 
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 30ms, vehicles TOT 0 ACT 0 BUF 0)                     


## Test the trained DQN policy on emergency scenerio

In [5]:
from stable_baselines3 import DQN

model = DQN.load("agents/dqn_emergency")

env = EmergencySumoEnvironment(
    net_file="network_details/intersection.net.xml",
    route_files=route_files,
    out_csv_name=f'outputs/test/Emergency/DQN',
    min_green=5,
    yellow_time=5,
    delta_time=10,
    use_gui=False,
    num_seconds=5400,
    observation_class=EmergencyObservationFunction,
    reward_fn=emergency_reward
)

obs, info = env.reset()
done = False

while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

obs, info = env.reset()
env.close()

/home/saksham/miniconda3/envs/traffic-light-control/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/saksham/miniconda3/envs/traffic-light-control/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 12ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 400 ACT 0 BUF 0)                  tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 19ms, vehicles TOT 392 ACT 4 BUF 0)                 
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 22ms, vehicles TOT 0 ACT 0 BUF 0)                     
